<a href="https://colab.research.google.com/github/soumabkargit/intro-to-machine-learning/blob/master/VisualizingTensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install tf-nightly

# import packages
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras import Model
from sklearn.preprocessing import StandardScaler
from tensorboard import notebook

 # View open TensorBoard instances

notebook.list()

# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

# load dataset and split in train and test sets
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

# standardize the dataset
scaler_X_train = StandardScaler().fit(X_train)
scaler_X_test = StandardScaler().fit(X_test)
X_train = scaler_X_train.transform(X_train)
X_test = scaler_X_test.transform(X_test)

# reshape y-data to become column vector
y_train = np.reshape(y_train, [-1, 1])
y_test = np.reshape(y_test, [-1, 1])

# build the linear model
class LinearRegressionModel(Model):
  def __init__(self):
    super(LinearRegressionModel, self).__init__()
    # initialize weight and bias variables
    self.weight = tf.Variable(
        initial_value = tf. random.normal(
            [13, 1], dtype=tf.float64),
        trainable=True)
    self.bias = tf.Variable(initial_value = tf.constant(
        1.0, shape=[], dtype=tf.float64), trainable=True)

  def call(self, inputs):
    return tf.add(tf.matmul(inputs, self.weight), self.bias)

model = LinearRegressionModel()

# parameters
batch_size = 32
learning_rate = 0.01

# use tf.data to batch and shuffle the dataset
train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(len(X_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_rmse = tf.keras.metrics.RootMeanSquaredError(name='train_rmse')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_rmse = tf.keras.metrics.RootMeanSquaredError(name='test_rmse')

# use tf.GradientTape to train the model
@tf.function
def train_step(inputs, labels):
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_rmse(labels, predictions)

@tf.function
def test_step(inputs, labels):
  predictions = model(inputs)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_rmse(labels, predictions)

  # Clear any logs from previous runs
!rm -rf ./logs/

# set up summary writers to write the summaries to disk in a different logs directory
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

num_epochs = 1000

for epoch in range(num_epochs):
  for train_inputs, train_labels in train_ds:
    train_step(train_inputs, train_labels)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('rmse', train_rmse.result(), step=epoch)

  for test_inputs, test_labels in test_ds:
    test_step(test_inputs, test_labels)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('rmse', test_rmse.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, RMSE: {}, Test Loss: {}, Test RMSE: {}'

  if ((epoch+1) % 100 == 0):
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_rmse.result(),
                           test_loss.result(),
                           test_rmse.result()))
    
  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_rmse.reset_states()
  test_rmse.reset_states()

     |████████████████████████████████| 402.8MB 34kB/s 


No known TensorBoard instances running.


Epoch 100, Loss: 22.037517547607422, RMSE: 4.724170207977295, Test Loss: 29.118572235107422, Test RMSE: 4.610310077667236
Epoch 200, Loss: 22.01976776123047, RMSE: 4.722591400146484, Test Loss: 29.16248321533203, Test RMSE: 4.618571758270264
Epoch 300, Loss: 22.01911163330078, RMSE: 4.722567558288574, Test Loss: 29.1754150390625, Test RMSE: 4.620066165924072
Epoch 400, Loss: 22.019094467163086, RMSE: 4.722574234008789, Test Loss: 29.178226470947266, Test RMSE: 4.6203837394714355
Epoch 500, Loss: 22.019094467163086, RMSE: 4.722576141357422, Test Loss: 29.178810119628906, Test RMSE: 4.620449066162109
Epoch 600, Loss: 22.01909637451172, RMSE: 4.722577095031738, Test Loss: 29.17892837524414, Test RMSE: 4.620462417602539
Epoch 700, Loss: 22.01909637451172, RMSE: 4.72257661819458, Test Loss: 29.178953170776367, Test RMSE: 4.620465278625488
Epoch 800, Loss: 22.01909637451172, RMSE: 4.72257661819458, Test Loss: 29.178958892822266, Test RMSE: 4.6204657554626465
Epoch 900, Loss: 22.0190963745117